In [5]:
# Cell 1: Imports and Base Classes
import os
import pickle
import time
import requests
import numpy as np
from collections import defaultdict
from dataclasses import dataclass
from typing import List, Optional, Dict
from concurrent.futures import ThreadPoolExecutor
import logging
from functools import lru_cache

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class Response:
    """Enhanced response structure with multi-agent processing metadata"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    critique: Optional[str] = None
    refinement: Optional[str] = None
    iterations: int = 0

class OllamaClient:
    """Handles communication with Ollama API"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", 
                 base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self._check_model()
        
    def _check_model(self):
        """Verify model exists in Ollama"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            response.raise_for_status()
            available_models = [model['name'] for model in response.json().get('models', [])]
            model_base = self.model_name.split('/')[-1].split(':')[0]
            
            if model_base not in available_models:
                logger.warning(f"Model {model_base} not found. Attempting to pull...")
                self._pull_model()
        except Exception as e:
            logger.error(f"Error checking model: {e}")
            raise
                
    def _pull_model(self):
        """Pull model from Hugging Face"""
        try:
            response = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                timeout=600
            )
            response.raise_for_status()
            logger.info(f"Successfully pulled model {self.model_name}")
        except Exception as e:
            logger.error(f"Failed to pull model: {e}")
            raise

    def generate(self, prompt: str) -> str:
        """Generate response from Ollama model"""
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json={
                    "model": self.model_name,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": 0.7,
                        "top_p": 0.9,
                        "top_k": 40
                    }
                },
                timeout=30
            )
            response.raise_for_status()
            return response.json().get("response", "Error: Empty response from API")
        except Exception as e:
            logger.error(f"Ollama API error: {e}")
            return f"Error: Failed to generate response - {str(e)}"

class BaseAgent:
    """Base class for all agents with common functionality"""
    def __init__(self, client: OllamaClient):
        self.client = client

    def generate_response(self, prompt: str) -> Response:
        """Base method for generating responses"""
        start_time = time.time()
        try:
            text = self.client.generate(prompt)
            error = text.startswith("Error:")
        except Exception as e:
            text = f"Error: {e}"
            error = True
        return Response(
            text=text,
            timestamp=start_time,
            error=error,
            processing_time=time.time() - start_time
        )

# Cell 2: Core Agents
class GeneratorAgent(BaseAgent):
    """Generates initial responses with contextual awareness"""
    def generate(self, base_prompt: str, context: str) -> Response:
        prompt = f"""Generate a comprehensive response using context:
        {context}
        Query: {base_prompt}
        Structured response:"""
        return self.generate_response(prompt)

class CriticAgent(BaseAgent):
    """Analyzes responses for quality and accuracy"""
    def critique(self, prompt: str, response: str) -> Response:
        critique_prompt = f"""Critique this response:
        Query: {prompt}
        Response: {response}
        List strengths/weaknesses/suggestions:"""
        return self.generate_response(critique_prompt)

class RefinerAgent(BaseAgent):
    """Improves responses based on feedback"""
    def refine(self, prompt: str, response: str, critique: str) -> Response:
        refine_prompt = f"""Improve this response:
        Original Query: {prompt}
        Original Response: {response}
        Critique: {critique}
        Enhanced Response:"""
        return self.generate_response(refine_prompt)

# Cell 3: Enhanced Agents
class ContextManagerAgent(BaseAgent):
    """Manages and enhances conversation context"""
    def enhance_context(self, query: str, current_context: str) -> tuple:
        start_time = time.time()
        prompt = f"""Enhance context with relevant information:
        Current Context: {current_context}
        Query: {query}
        Enhanced Context:"""
        response = self.generate_response(prompt)
        return response.text, time.time() - start_time

class ValidatorAgent(BaseAgent):
    """Validates responses against knowledge base"""
    def validate(self, response: str, knowledge_base: dict) -> tuple:
        start_time = time.time()
        prompt = f"""Validate response against knowledge:
        Knowledge: {str(knowledge_base)[:1000]}
        Response: {response}
        List inaccuracies:"""
        validation = self.generate_response(prompt)
        return validation.text, time.time() - start_time

class EmotionSummarizerAgent(BaseAgent):
    """Combined emotion analysis and summarization"""
    def analyze_and_summarize(self, text: str) -> tuple:
        start_time = time.time()
        emotion_prompt = f"""Analyze emotional tone (1-5):
        Text: {text}
        Analysis:"""
        emotion_analysis = self.generate_response(emotion_prompt)
        
        summary_prompt = f"""Summarize under 100 words:
        {text}
        Summary:"""
        summary = self.generate_response(summary_prompt)
        return emotion_analysis.text, summary.text, time.time() - start_time

# Cell 4: System Implementation
class EnhancedMultiAgentSystem:
    """Coordinated multi-agent system with quality control"""
    def __init__(self, model_path: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L",
                 pickle_file: str = "responses.pkl"):
        self.pickle_file = pickle_file
        self.responses: List[Response] = self._load_responses()
        self.executor = ThreadPoolExecutor(max_workers=4)
        self.client = OllamaClient(model_path)
        self.knowledge_base = {}
        self.metrics = SystemMetrics()
        
        # Initialize agents
        self.generator = GeneratorAgent(self.client)
        self.critic = CriticAgent(self.client)
        self.refiner = RefinerAgent(self.client)
        self.context_manager = ContextManagerAgent(self.client)
        self.validator = ValidatorAgent(self.client)
        self.emotion_summarizer = EmotionSummarizerAgent(self.client)

    def _load_responses(self) -> List[Response]:
        """Load responses from storage"""
        try:
            if os.path.exists(self.pickle_file):
                with open(self.pickle_file, "rb") as f:
                    data = pickle.load(f)
                    return data if isinstance(data, list) else []
        except Exception as e:
            logger.error(f"Error loading responses: {e}")
        return []

    @lru_cache(maxsize=1000)
    def find_similar_insights(self, query: str, top_k: int = 3) -> str:
        """Find similar historical responses"""
        if not self.responses:
            return ""
            
        query_words = set(query.lower().split())
        scores = []
        
        for response in self.responses:
            if response.error:
                continue
            response_words = set(response.text.lower().split())
            union = query_words | response_words
            similarity = len(query_words & response_words)/len(union) if union else 0
            scores.append((response.text, similarity))
            
        return "\n".join([f"- {t[:200]}..." for t,_ in sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]])

    def execute_quality_pipeline(self, prompt: str) -> Response:
        """Full processing pipeline"""
        if not prompt.strip():
            return Response(text="Error: Empty input", timestamp=time.time(), error=True)

        # Context enhancement
        base_context = self.find_similar_insights(prompt)
        enhanced_context, ctx_time = self.context_manager.enhance_context(prompt, base_context)
        self.metrics.update('ContextManager', time=ctx_time)
        
        # Generation
        gen_response = self.generator.generate(prompt, enhanced_context)
        self.metrics.update('Generator', length=len(gen_response.text))
        if gen_response.error:
            return gen_response
        
        # Critique & Refinement
        critique = self.critic.critique(prompt, gen_response.text)
        self.metrics.update('Critic', issues=len(critique.text.split('\n')))
        refined_response = self.refiner.refine(prompt, gen_response.text, critique.text)
        self.metrics.update('Refiner', iterations=1)
        
        # Validation
        validation, val_time = self.validator.validate(refined_response.text, self.knowledge_base)
        self.metrics.update('Validator', time=val_time, issues=validation.count('inaccuracy'))
        
        # Secondary refinement
        if "serious inaccuracies" in validation.lower():
            refined_response = self.refiner.refine(prompt, refined_response.text, validation)
            self.metrics.update('Refiner', iterations=1)
        
        # Emotion & Summary
        emotion, summary, es_time = self.emotion_summarizer.analyze_and_summarize(refined_response.text)
        self.metrics.update('EmotionAnalyzer', score=self._parse_emotion(emotion))
        self.metrics.update('Summarizer', compression=len(summary)/len(refined_response.text))
        
        # Update knowledge base
        self._update_knowledge(prompt, refined_response.text)
        
        return Response(
            text=summary,
            timestamp=time.time(),
            processing_time=gen_response.processing_time + 
                          critique.processing_time + 
                          refined_response.processing_time +
                          ctx_time + val_time + es_time,
            critique=critique.text,
            refinement=refined_response.text,
            iterations=1
        )

    def _parse_emotion(self, text: str) -> float:
        """Extract emotion score from analysis"""
        try:
            return float(text.split(':')[-1].strip())
        except:
            return 3.0  # Default neutral

    def _update_knowledge(self, query: str, response: str):
        """Update knowledge base"""
        key = hash(query[:50])
        self.knowledge_base[key] = {
            'query': query,
            'response': response,
            'timestamp': time.time()
        }

    def _save_response(self, response: Response):
        """Save validated responses"""
        try:
            self.responses.append(response)
            with open(self.pickle_file, "wb") as f:
                pickle.dump(self.responses, f)
        except Exception as e:
            logger.error(f"Error saving response: {e}")

class SystemMetrics:
    """Performance tracking system"""
    def __init__(self):
        self.metrics = defaultdict(lambda: {'count': 0, 'total': 0.0})
        self.quality = {
            'emotion_scores': [],
            'compression_rates': [],
            'validation_issues': 0
        }
        
    def update(self, agent: str, **kwargs):
        self.metrics[agent]['count'] += 1
        for key, value in kwargs.items():
            if key in ['time', 'length']:
                self.metrics[agent]['total'] += value
            elif key == 'score':
                self.quality['emotion_scores'].append(value)
            elif key == 'compression':
                self.quality['compression_rates'].append(value)
            elif key == 'issues':
                self.quality['validation_issues'] += value

    def report(self):
        """Generate metrics report"""
        print("=== System Metrics ===")
        for agent, data in self.metrics.items():
            avg = data['total']/data['count'] if data['count'] else 0
            print(f"{agent}: {data['count']} calls, Avg: {avg:.2f}")
        
        print("\nQuality Metrics:")
        print(f"Avg Emotion: {np.mean(self.quality['emotion_scores']) if self.quality['emotion_scores'] else 0:.1f}")
        print(f"Avg Compression: {np.mean(self.quality['compression_rates']) if self.quality['compression_rates'] else 0:.1%}")
        print(f"Total Issues Found: {self.quality['validation_issues']}")

# Cell 5: Testing Framework
def test_pipeline():
    """Comprehensive test suite"""
    system = EnhancedMultiAgentSystem()
    tests = [
        ("Explain quantum computing basics", "normal"),
        ("", "empty input"),
        ("What's the capital of France?", "factual"),
        ("I'm worried about AI safety", "emotional")
    ]

    for query, test_type in tests:
        print(f"\n=== Testing {test_type} ===")
        start = time.time()
        try:
            response = system.execute_quality_pipeline(query)
            
            assert isinstance(response, Response), "Invalid response type"
            if test_type == "empty input":
                assert response.error, "Should handle empty input"
            else:
                assert len(response.text) > 20, "Response too short"
                assert response.processing_time < 15, "Processing too slow"
            
            print(f"Passed in {time.time()-start:.1f}s")
        except Exception as e:
            print(f"Test failed: {str(e)}")
    
    print("\nFinal Metrics Report:")
    system.metrics.report()

if __name__ == "__main__":
    test_pipeline()

2025-02-19 14:27:31,483 - WARNING - Model Gemmasutra-Mini-2B-v1-GGUF not found. Attempting to pull...
2025-02-19 14:27:31,854 - INFO - Successfully pulled model hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L



=== Testing normal ===
Test failed: Processing too slow

=== Testing empty input ===
Passed in 0.0s

=== Testing factual ===


2025-02-19 14:31:55,751 - ERROR - Ollama API error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)


Test failed: Processing too slow

=== Testing emotional ===


2025-02-19 14:32:27,345 - ERROR - Ollama API error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)
2025-02-19 14:32:59,640 - ERROR - Ollama API error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)


Test failed: Processing too slow

Final Metrics Report:
=== System Metrics ===
ContextManager: 3 calls, Avg: 23.26
Generator: 3 calls, Avg: 1646.00
Critic: 2 calls, Avg: 0.00
Refiner: 2 calls, Avg: 0.00
Validator: 2 calls, Avg: 16.70
EmotionAnalyzer: 2 calls, Avg: 0.00
Summarizer: 2 calls, Avg: 0.00

Quality Metrics:
Avg Emotion: 3.0
Avg Compression: 32.3%
Total Issues Found: 40
